**pixyz  
last update 2022 03 12  
ゆっくりしていってね！**

version8: article_id(購入した商品)、pred_id(購入すべき商品)、confidence(購入数)をまとめたdataframeをpickleで作成しました。SHORT_CUTを追加しました。  
version10:客の年齢層毎のitem_of_other_costomersを作成しました。

<img src="https://4.bp.blogspot.com/-6jLig_Zuhyk/UUhH8z560_I/AAAAAAAAO6A/lFCDFT8S1FM/s400/shopping_fasion.png" width=200>

**霊夢:H&Mコンペ、客の数も商品の数も多いし、どうやって学習させればいいかわからないなあ**

**魔理沙:「この商品を買った客は、あの商品もよく買ってるぞ！」って言うデータがあればいいのにな。**

**霊夢:無ければ作るしかない！**

**魔理沙:「ある客が買った商品一覧」の辞書と「製品を買った客一覧」の辞書があれば作れそうだな。**

**魔理沙:ということで、「ある客が買った商品一覧」、「製品を買った客一覧」の辞書データと「ある商品を買った人が他に買っている商品ランキング」データをまとめたデータセットを作ったぜ。**

**霊夢:良ければ学習の参考にしてね！**

<br>

**Reimu: H&M competition, there are a lot of customers and products, so I don't know how to learn.**

**Marisa: I wish I had the data that "customers who bought this product often buy that product too!"**

**Reimu: If you don't have it, you have to make it!**

**Marisa: It seems that you can make it if you have a dictionary of "list of products bought by a certain customer" and a dictionary of "list of customers who bought products".**

**Marisa: So, the dictionary data of "List of products bought by a certain customer", "List of customers who bought a product" and "Ranking of products bought by a person who bought a certain product" are summarized. I made a new data set.**

**Reimu: If you like, use it as a reference for learning!**


In [ ]:
SHORT_CUT = True

# Import

In [ ]:
import numpy as np
import pandas as pd

from math import sqrt
from pathlib import Path
from tqdm import tqdm
import json
tqdm.pandas()

In [ ]:
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)


In [ ]:
!cp ../input/hm-dictionary/items_of_other_costomers.json ./items_of_other_costomers.json
!cp ../input/hm-dictionary/items_of_other_costomers.pkl ./items_of_other_costomers.pkl

# Path Setting

In [ ]:
data_path = Path('../input/h-and-m-personalized-fashion-recommendations/')
dict_path = Path('../input/hm-dictionary/')

# Data Load

In [ ]:
df = pd.read_csv(data_path / 'transactions_train.csv',
                 usecols = ['customer_id', 'article_id'],
                 dtype={'article_id': int})
df_articles = pd.read_csv(data_path / "articles.csv")

version 10 追記  
**魔理沙:年齢層ごとに別のランキングを作りたいので、客がどの年齢層に所属しているかを示すデータを作っていくよ。**

In [ ]:
df_customers = pd.read_csv(data_path / 'customers.csv', usecols=['customer_id', 'age'],index_col = 'customer_id')
df_customers.head()

In [ ]:
listBin = [-1, 19, 29, 39, 49, 59, 69, 119]
df_customers['age_bins'] = pd.cut(df_customers['age'], listBin).astype(str)
df_customers.head()

In [ ]:
cus_agebins = df_customers['age_bins'].astype(str)

In [ ]:
listUniBins = df_customers['age_bins'].unique().tolist()

# Make dictionary from customer to article

**魔理沙:まずは、「ある客が買った商品一覧」の辞書を作っていくぜ。**

**Marisa: First, let's make a dictionary of "a list of products bought by a customer".**

In [ ]:
if SHORT_CUT:
    with open(dict_path / "dict_c_a.json", mode="r") as f:
        ds_dict_c_a = json.load(f)        
else:
    ds_dict_c_a={}
    for i in tqdm(range(len(df))):
        customer_id=df.loc[i,"customer_id"]
        article_id=df.loc[i,"article_id"]
        if customer_id in ds_dict_c_a:
            ds_dict_c_a[customer_id]=ds_dict_c_a[customer_id]+[article_id]        
        else:
            ds_dict_c_a[customer_id]=[article_id]    

# Make dictionary from article to customer

**魔理沙:次に、「ある商品を買った客一覧」の辞書を作っていくぜ。**

**Marisa: Next, let's make a dictionary of "list of customers who bought a certain product".**

In [ ]:
# if SHORT_CUT:
#     with open(dict_path / 'dict_a_c.json', mode="r") as f:
#         ds_dict_a_c = json.load(f)
# else:
#     ds_dict_a_c={}
#     for i in tqdm(range(len(df))):
#         article_id=df.loc[i,"article_id"]
#         ds_dict_a_c[int(article_id)]=[]
#     for i in tqdm(range(len(df))):
#         customer_id=df.loc[i,"customer_id"]
#         article_id=df.loc[i,"article_id"]
#         ds_dict_a_c[int(article_id)]=ds_dict_a_c[int(article_id)]+[customer_id]

In [ ]:
ds_dict_a_c = {listUniBins[i]:{} for i in range(len(listUniBins))}

In [ ]:
for i in tqdm(range(len(df))):
    customer_id=df.loc[i,"customer_id"]
    age_bin = cus_agebins[customer_id] 
    article_id=df.loc[i,"article_id"]
    ds_dict_a_c[age_bin][int(article_id)]=[]
for i in tqdm(range(len(df))):
    customer_id=df.loc[i,"customer_id"]
    age_bin = cus_agebins[customer_id] 
    article_id=df.loc[i,"article_id"]
    ds_dict_a_c[age_bin][int(article_id)]+=[customer_id]

# Save as json file

**霊夢:jsonfileにして保存するよ。**

**Reimu: Save it as a jsonfile.**

In [ ]:
with open("dict_c_a.json", mode="w") as f:
    ds_dict_c_a = json.dumps(ds_dict_c_a,cls = MyEncoder)
    f.write(ds_dict_c_a)
with open("dict_a_c.json", mode="w") as f:
    ds_dict_a_c = json.dumps(ds_dict_a_c,cls = MyEncoder)
    f.write(ds_dict_a_c)

In [ ]:
ds_dict_a_c ={}
ds_dict_c_a ={}    
with open('dict_a_c.json', mode="r") as f:
    ds_dict_a_c = json.load(f)
with open("dict_c_a.json", mode="r") as f:
    ds_dict_c_a = json.load(f)        

In [ ]:
print(ds_dict_c_a["00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657"])

# Make dict article_id to index and index to article_id

**魔理沙:買った商品集計リストを作成しやすくするために、article_idを空間圧縮していくぜ。**

**Marisa: I'm going to spatially compress the article_id to make it easier to create a list of the items I bought.**

In [ ]:
df_a_i = {}
df_i_a = {}
for i in range(len(df_articles)):
    df_a_i[df_articles.loc[i,"article_id"]] = i
    df_i_a[i] = df_articles.loc[i,"article_id"]

# Make dictionary items of other customers top 100

**魔理沙：そしたらいよいよ、「ある商品を買った人が他に買っている商品ランキング」を作成していくぜ。**

**Marisa: Then, finally, we will create a "ranking of products that people who bought one product are buying elsewhere".**

In [ ]:
# ranking = {}
# for articl,coslist in tqdm(ds_dict_a_c.items()):
#     df_count = np.zeros(len(df_articles))
#     for costomer in coslist:
#         for x in ds_dict_c_a[costomer]:
#             df_count[df_a_i[x]] += 1
#     df_count=np.argsort(df_count)
#     df_count = df_count[-1::-1]
#     ranking[int(articl)] = []
#     for i in range(100):
#         ranking[int(articl)]=ranking[int(articl)]+[df_i_a[df_count[i]]]
#     del df_count

In [ ]:
cols=["article_id","pred_id","confidence"]
table = pd.DataFrame(index=[], columns=cols)

In [ ]:
# ##test
# coslist = ds_dict_a_c['728162001']
# count = [0]*len(df_articles)
# for costomer in coslist:
#     for x in ds_dict_c_a[costomer]:
#         count[df_a_i[x]] += 1
            
# pred_list = sorted(range(len(df_articles)),key = lambda k:count[k],reverse = True)[:100]
# for i in range(len(pred_list)):
#     pred_list[i] = df_i_a[pred_list[i]]
# conf_list = sorted(count,reverse=True)[:100]

# print(pred_list)
# print(conf_list)

In [ ]:
for uniBin in listUniBins:
    if uniBin == 'nan': continue
    print(uniBin)
    article_id = []
    pred_id = []
    confidence = []
    for articl,coslist in tqdm(ds_dict_a_c[uniBin].items()):
        count = [0]*len(df_articles)    
        for costomer in coslist:
            for x in ds_dict_c_a[costomer]:
                count[df_a_i[x]] += 1
            
        art_list = [articl]*100
        pred_list = sorted(range(len(df_articles)),key = lambda k:count[k],reverse = True)[:100]
        for i in range(len(pred_list)):
            pred_list[i] = df_i_a[pred_list[i]]
        conf_list = sorted(count,reverse=True)[:100]

        article_id.extend(art_list)
        pred_id.extend(pred_list)
        confidence.extend(conf_list)
        del art_list
        del pred_list
        del conf_list
    
    table = pd.DataFrame(list(zip(article_id,pred_id,confidence)),columns = cols)
    table.to_pickle(f"items_of_other_costomers_{uniBin}.pkl")

In [ ]:
# table = pd.DataFrame(list(zip(article_id,pred_id,confidence)),columns = cols)

In [ ]:
table.head()

# Save as pickle file

**霊夢:最後にセーブして完了だね。**

**Reimu: Finally save and you're done.**

In [ ]:
# with open("items_of_other_costomers.json", mode="w") as f:
#         ranking = json.dumps(ranking,cls = MyEncoder)
#         f.write(ranking)        

In [ ]:
# table.to_pickle("items_of_other_costomers.pkl")

In [ ]:
with open("items_of_other_costomers.json", mode="r") as f:
        ds_dict = json.load(f)

**魔理沙:これ使ったらいいスコアでるといいなあ。**

**霊夢:次回このデータを使ってpredictionしてみよう！**

<br>

**Marisa: I hope you get a good score if you use this.**

**Reimu: Let's use this data for prediction next time!**

<br>

**読んでくれてありがとうございました！！よければvoteをよろしくお願いします！！**

**Thank you for reading! !! If you like, please vote!!!**

<img src="https://1.bp.blogspot.com/-PNcKwFw1PpM/U1T3oDIr9CI/AAAAAAAAfT4/gEn86X8Ppx0/s400/figure_goodjob.png" width=200>